In [151]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pickle
from multiprocessing.pool import ThreadPool
from datetime import datetime
from functools import lru_cache

In [2]:
gipfel_base_path = "http://db-sandsteinklettern.gipfelbuch.de/weg.php?gipfelid={}"
route_base_path = "http://db-sandsteinklettern.gipfelbuch.de/komment.php?wegid={}"

In [122]:
def timout(tries):
    """Decorator"""
    def real_decorator(function):
        def wrapper(*args, **kwargs):
            for j in range(tries):
                try:
                    return function(*args, **kwargs)
                    break
                except:
                    print("no connection")
                    time.sleep(5)
            if j == (tries - 1):
                raise Exception("failed on all trys")
        return wrapper
    return real_decorator

In [153]:
@timout(3)
def get_route_info(internal_id):
    infos = {}
    
    resp = requests.get(route_base_path.format(internal_id))
    soup = BeautifulSoup(resp.content, "lxml")
    
    links = soup.find_all("a")
    
    summit_tag_list = list(filter(lambda x: "weg.php?gipfelid" in str(x), links))
    if len(summit_tag_list) == 0:
        return "Da war wohl nichts zu holen"
    
    summit_tag = summit_tag_list[0]
    summit_id = summit_tag.attrs["href"].split("=")[1]
    
    infos['area'] = next(filter(lambda x: "gebiet" in str(x), links)).text
    infos['sector'] = next(filter(lambda x: "sektor" in str(x), links)).text
    infos['summit_name'] = summit_tag.text
    infos['summit_id'] = str(get_summit_id(summit_id))
    #infos['summit_internal_id'] = str(summit_id)
    
    route_name_tag = summit_tag.next_element.next_element.next_element
    route_name = re.match("\d?(\*?.*)", route_name_tag.strip("\n")).groups()[0]
    infos['route_name'] = route_name
    
    diff_tag = route_name_tag.find_next("a").next_element.next_element
    infos['diff'] = diff_tag
    
    first_ascent = diff_tag.find_next("a").next_element.next_element
    infos['first_ascent'] = first_ascent
    
    route_descr = first_ascent.find_next("a").find_next("a").previous_element.previous_element
    
    if route_descr.startswith("Kletterei"):
        route_descr = route_descr.find_next("a").find_next("a").previous_element.previous_element
        
    infos['descr'] = route_descr
    
    for info_key in infos:
        infos[info_key] = infos[info_key].strip("\n").strip()
    
    return infos

In [141]:
get_route_info(100)

{'area': 'Adersbach',
 'descr': 'Von schartenseitig vorgelagerten Blöcken links Wand zu Hangelriß, diesen später Riß u. Kamin zu Absatz. (Mitte Sanduhr). rechts Wand zu Vorgipfel, Übertritt Wand z.G.',
 'diff': 'VIIa',
 'first_ascent': 'Peter Die\x9aka, L.Páleníček, M.Ent 16.09.1967',
 'route_name': '*Beova',
 'sector': 'Königreich / Království',
 'summit_internal_id': '102',
 'summit_name': 'Schloß / Zámek'}

In [152]:
@timout(3)
@lru_cache(maxsize=1500)
def get_summit_id(internal_id):
    resp = requests.get(gipfel_base_path.format(internal_id))
    soup = BeautifulSoup(resp.content, "lxml")
    
    h2_tags = soup.find_all("h2")
    
    g_id = str(h2_tags[0].next_element).strip()
    try:
        g_id = int(g_id)
    except:
        pass
    
    return g_id

In [10]:
get_summit_id(102)

64

# Test

In [179]:
start = datetime.now()

pool = ThreadPool(50)

total = pool.map(get_route_info, range(60000, 100000))

print(datetime.now() - start)

no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no connection
no con

In [180]:
len(total)

40000

In [181]:
pickle.dump(total, open("./60000-100000.pkl", "wb"))

# Schwierigkeiten Splitten

In [209]:
diffs = ["9b XIa", "VIIa", "2", "VIIIc", "VIIIc !", "VIIIc RP IXa", "RP Xb", "2/V", "VIIa/3" "V (VIIb)"]

- erstbegehung eg.: 9a
- normal
- unterstützung
- sprung
- af 
- rp
- zusatz eg.: anstr